In [54]:
import requests

In [55]:
csUrl = "https://www.cheapshark.com/api/1.0/"

def supCheapSharkExtractor(path,**kwargs) -> list:
    """
    Comentário teste...
    """ 
    send = csUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    print(send)

    return requests.get(send).json()

def getGameByTheme(title:str = None,size:int = 60) -> list:
    """
    Retorna uma lista de jogos baseados em um tema.
    """
    response = supCheapSharkExtractor(path="games?",title=title,size=size)

    response_id = []

    for i in response: 
        if i['steamAppID'] != None: response_id.append(i)

    return response_id

In [78]:
stUrl = "https://store.steampowered.com/api/"

def supSteamPoweredExtractor(path,**kwargs) -> list:
    send = stUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    print(send)

    return requests.get(send).json()

def getGameOverview(appids:str = None):
    response = supSteamPoweredExtractor(path="appdetails?",appids=appids,cc="pt-br")

    return response

In [130]:
from env import env

adUrl = "https://api.isthereanydeal.com/"

def supIsThereAnyDealExtractor(path,**kwargs) -> list: # to do: add key

    send = adUrl + path

    for key,value in kwargs.items():
        if value != None: send += f"{key}={value}&"

    print(send)

    return requests.get(send).json()

def getGameLowest(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/overview/?",key=env("AnyDealKey"),country="BR",shop="steam",ids=f"app/{gameid}")
    return response

def getGamePlains(gameid:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/plain/id/?",key=env("AnyDealKey"),shop="steam",ids=f"app/{gameid}")
    return response

def getGamePrices(gamePlain:str = None):
    response = supIsThereAnyDealExtractor(path="v01/game/prices/?",key=env("AnyDealKey"),plains=gamePlain,country="BR")
    return response

## Escolha do tema e jogo

In [141]:
response = getGameByTheme(title="red dead redemption")

response_id = []

for i in response: 
    if i['steamAppID'] != None: response_id.append(i)

for i in response_id:
    print(i)

https://www.cheapshark.com/api/1.0/games?title=red dead redemption&size=60&
{'gameID': '206516', 'steamAppID': '1174180', 'cheapest': '25.99', 'cheapestDealID': 'kZsu5wC0oIzToXYWmjh7WNag3qgqRnAUEfZSUC5hX30%3D', 'external': 'Red Dead Redemption 2', 'internalName': 'REDDEADREDEMPTION2', 'thumb': 'https://cdn.cloudflare.steamstatic.com/steam/apps/1174180/capsule_sm_120.jpg?t=1695140956'}


## Retorno do overview

In [142]:
gameid = "1174180"
game_overview = getGameOverview(appids=gameid)

https://store.steampowered.com/api/appdetails?appids=1174180&cc=pt-br&


In [143]:
print(game_overview)

{'1174180': {'success': True, 'data': {'type': 'game', 'name': 'Red Dead Redemption 2', 'steam_appid': 1174180, 'required_age': 0, 'is_free': False, 'detailed_description': '<h1>Ultimate Edition</h1><p><img src="https://cdn.akamai.steamstatic.com/steam/apps/1174180/extras/Ultimate_615.jpg?t=1695140956" /><br>The Red Dead Redemption 2:  Ultimate Edition delivers all of the Story Mode content from the Special Edition plus additional content for online including Bonus Outfits for your online Character, Rank Bonuses, Black Chestnut Thoroughbred and free access to the Survivor Camp Theme.<br><br>Plus get free access to Additional Weapons in online.</p><br><h1>About the Game</h1>America, 1899.<br><br>Arthur Morgan and the Van der Linde gang are outlaws on the run. With federal agents and the best bounty hunters in the nation massing on their heels, the gang must rob, steal and fight their way across the rugged heartland of America in order to survive. As deepening internal divisions threaten

In [145]:
game_overview[gameid]["data"]['is_free']

game_overview[gameid]["data"]['detailed_description']

game_overview[gameid]["data"]['developers']
game_overview[gameid]["data"]['publishers']

game_overview[gameid]["data"]['price_overview']['final_formatted']

game_overview[gameid]['data']['genres']

game_overview[gameid]['data']['release_date']

#game_overview[gameid]['data']

{'coming_soon': False, 'date': '5 Dec, 2019'}

## Retorno menor preço atual e histórico

In [146]:
game_lowest = getGameLowest(gameid)

https://api.isthereanydeal.com/v01/game/overview/?key=48cf3848edaf5a53d702c5b056b0dcb4be35df34&country=BR&shop=steam&ids=app/1174180&


In [147]:
from datetime import datetime

print(type(game_lowest['data'][f"app/{gameid}"]))

game_lowest['data'][f"app/{gameid}"]['price'] # today´s lowest
game_lowest['data'][f"app/{gameid}"]['price']['store']


game_lowest['data'][f"app/{gameid}"]['lowest'] # historical lowest
game_lowest['data'][f"app/{gameid}"]['lowest']['store']

game_lowest['data'][f"app/{gameid}"]['urls']['history'] # price history site

game_lowest['data'][f"app/{gameid}"]['lowest']['recorded']

data_formatada = datetime.utcfromtimestamp(game_lowest['data'][f"app/{gameid}"]['lowest']['recorded']).strftime('%d-%m-%Y') # dia do menor preço histórico

print(data_formatada)

<class 'dict'>
<class 'str'>


## Retorno todos os preços do jogo

In [148]:
plainResponse = getGamePlains(gameid=gameid)

game_plain = plainResponse['data'][f"app/{gameid}"]

print(game_plain)

pricesResponse = getGamePrices(game_plain)

https://api.isthereanydeal.com/v01/game/plain/id/?key=48cf3848edaf5a53d702c5b056b0dcb4be35df34&shop=steam&ids=app/1174180&
reddeadredemptionii
https://api.isthereanydeal.com/v01/game/prices/?key=48cf3848edaf5a53d702c5b056b0dcb4be35df34&plains=reddeadredemptionii&country=BR&


In [149]:
#pricesResponse['data'][game_plain]['list']

for i in pricesResponse['data'][game_plain]['list']:
    print(i['price_old'],i['price_new'],i['shop']['name'])

292.776 126.842 GameBillet
239 239 GreenManGaming
293.881 279.185 GamesPlanet US
292.787 292.787 Humble Store
293.881 293.881 Fanatical
299.9 299.9 Gamesload
299.9 299.9 Epic Game Store
299.9 299.9 Steam
299.9 299.9 GamersGate
299.9 299.9 Dreamgame
299.9 299.9 GreenManGaming
299.9 299.9 Nuuvem
299.9 299.9 2game
319.805 303.812 GamesPlanet FR
319.805 303.812 GamesPlanet DE
373.486 354.808 GamesPlanet UK
